In [77]:
# Going off the work of https://ieeexplore.ieee.org/document/8613389

Steps:

1. Model Formulation (With flexible dynamics)
2. Setup nonlinear equations and solver
3. Design observer
4. Composite controller design
5. Numerical simulation

Flexible modal frequency is assume to be: $\Omega$ = 0.3 rad/s and damping of 0.005

Moment of inertia in pitch axis is Iyy = 440 Kg*m^2

Fy = 1.278

Orbit:

Altitude: 36000 Km

Orbit rate: $\omega$<sub>0</sub> = 7.292e-5 rad/s

Disturbance torques:

w<sub>y</sub> = 10e-4 * cos($\omega$<sub>0</sub>t) Nm

Observer details:

Gain: ly = 2.85

State feedback: [kp, kd] = 5, 40

In [1]:
from sympy.physics.mechanics import *
from sympy import symbols, trigsimp
from numpy import deg2rad, rad2deg, array, zeros, linspace, pi
from scipy.integrate import odeint
from sympy.functions.elementary.trigonometric import sin, cos
from sympy import Matrix
from sympy import ones

from pydy.codegen.ode_function_generators import generate_ode_function

from sympy.physics.vector import init_vprinting
import matplotlib.pyplot as plt


In [4]:
# Define state vector

theta = dynamicsymbols('theta')
theta_d = dynamicsymbols('theta', 1)
theta_dd = dynamicsymbols('theta_d', 1)

# Define flexible dynamic states
eta = dynamicsymbols('eta')
eta_dot = dynamicsymbols('eta', 1)
eta_dd = dynamicsymbols('eta_dot', 1)

# Generalized speeds
u1 = dynamicsymbols('u1')
u1d = dynamicsymbols('u1', 1)

# For eta_dot
u2 = dynamicsymbols('u2')
u2d = dynamicsymbols('u2', 1)

# Reference Frames
Nrf = ReferenceFrame('Nrf')
Brf = Nrf.orientnew('Brf', 'Axis', [theta, Nrf.z])

Brf.set_ang_vel(Nrf, 0*Brf.x + 0*Brf.y + u1*Brf.z)

# Define locations
O = Point('O')
O.set_vel(Nrf, 0)

# Gather everything into lists
coordinates = [theta, eta]
speeds = [u1, u2]

kde = [u1 - theta_d, u2 - eta_dot]
kane = KanesMethod(Nrf, coordinates, speeds, kde)

I_x, I_y, I_z, m_sc = symbols('I_X I_Y I_Z m_sc') # Inertia of s/c and mass 

itd = inertia(Brf, I_x, I_y, I_z)

i1 = (itd, O)

sc_body = RigidBody('sc_body', O, Brf, m_sc, i1)

# Define forces and torques
Tw, Ty = dynamicsymbols('Tw Ty') # disturbance torques and control torques


t_Tw = (Brf, Tw*Brf.z)
t_Ty = (Brf, Ty*Brf.z)

loads = [t_Tw, t_Ty]
bodies = [sc_body]

fr, frstar = kane.kanes_equations(bodies, loads)
trigsimp(fr + frstar)

eqns = fr + frstar

In [3]:
F = symbols('F') # elastic coupling matrix
gamma, omega = symbols('gamma omega')
eqns[0]= eqns[0] - F*u2d
eqns[1] = u2d + 2*gamma*omega*u2 + omega**2*eta + F*u1d

In [88]:
eqns

Matrix([
[                   -F*Derivative(u2(t), t) - I_Z*Derivative(u1(t), t) + Tw(t) + Ty(t)],
[F*Derivative(u1(t), t) + 2*gamma*omega*u2(t) + omega**2*eta(t) + Derivative(u2(t), t)]])

In [89]:
from sympy import solve
f_eqn = ones(4,1)
f_eqn[0] = u1
f_eqn[1] = u2




f_eqn[2] = solve(eqns[0], u1d)
f_eqn[3] = solve(eqns[1], u2d)

In [10]:
from sympy import Derivative
eqns[0].subs({Derivative(u1, t): expression})

NameError: name 't' is not defined

In [90]:

f_eqn

Matrix([
[                                                          u1(t)],
[                                                          u2(t)],
[                  (-F*Derivative(u2(t), t) + Tw(t) + Ty(t))/I_Z],
[-F*Derivative(u1(t), t) - 2*gamma*omega*u2(t) - omega**2*eta(t)]])

In [84]:
type(f_eqn)

sympy.matrices.dense.MutableDenseMatrix

In [85]:
f_eqn.shape[0]

4

In [101]:
f_eqn

Matrix([
[                                           u1(t)],
[                                           u2(t)],
[                  (-F*q1(t) + Tw(t) + Ty(t))/I_Z],
[-F*q2(t) - 2*gamma*omega*u2(t) - omega**2*eta(t)]])

In [102]:
f_eqn.shape[0]

4

In [110]:

constants = [I_z, gamma, omega, F]
num_constants = array([440, 0.005, 0.3, 1.278])
rhs = generate_ode_function(right_hand_side = f_eqn, coordinates = coordinates, speeds = speeds, constants = constants)

t = linspace(0, 50, 500)
x0 = zeros(8)
rhs(x0, 0.0, num_constants)
#y = odeint(rhs, x0, t, args=(num_constants, ))



NameError: name 'q1' is not defined

In [70]:
rhs.__doc__

"Returns the derivatives of the states, i.e. numerically evaluates the right\nhand side of the first order differential equation.\n\nx' = f(x, t, p)\n\nParameters\n==========\nx : ndarray, shape(4,)\n    The state vector is ordered as such:\n        - theta(t)\n        - eta(t)\n        - u1(t)\n        - u2(t)\nt : float\n    The current time.\np : dictionary len(4) or ndarray shape(4,)\n    Either a dictionary that maps the constants symbols to their numerical\n    values or an array with the constants in the following order:\n        - I_Z\n        - gamma\n        - omega\n        - F\n\nReturns\n=======\ndx : ndarray, shape(4,)\n    The derivative of the state vector.\n\n"